# 감정분석 sentiment analysis

감성분석과 평균종가 비교 시각화 작업

In [67]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host = #호스트,
    port = 3306,
    user = #사용자,
    password = #비밀번호,
    database = "antking"
)
cursor = conn.cursor(buffered=True)

필요없는 심볼들 지우는 함수

In [ ]:
# Define unwanted symbols
unwanted_symbols = ["[", "]", "'"]

# Function to generate SQL query for removing unwanted symbols
def generate_update_query(symbol, replacement, table_name, column_name, id_range):
    query = f"""
    UPDATE {table_name}
    SET {column_name} = REPLACE({column_name}, '{symbol}', '{replacement}')
    WHERE article_id BETWEEN {id_range[0]} AND {id_range[1]};
    """
    return query

# Define the table and column to update, and the range of IDs
table_name = "keyword"
column_name = "keyword_set"
id_range = (1, 21297)

# Iterate over unwanted symbols and execute update queries
for symbol in unwanted_symbols:
    query = generate_update_query(symbol, "", table_name, column_name, id_range)
    cursor.execute(query)
    conn.commit()  # Commit the changes after each query

# Close the database connection
cursor.close()
conn.close()

In [56]:
# Define the SQL query to join the tables
query = """
SELECT
    a.id AS article_id,
    a.search_platform,
    a.search_keyword,
    a.pub_date,
    a.title,
    k.keyword_set,
    s.sentiment_from_ctext,
    s.sentiment_from_keyword,
    s.match_status
FROM
    article a
LEFT JOIN
    keyword k ON a.id = k.article_id
LEFT JOIN
    sentiment s ON a.id = s.article_id;
"""

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Define the column names based on the query
columns = [
    'article_id', 'search_platform', 'search_keyword', 'pub_date',
    'title', 'keyword_set',
    'sentiment_from_ctext', 'sentiment_from_keyword', 'match_status'
]

# Create a DataFrame from the results
df_combined = pd.DataFrame(results, columns=columns)

# Close the cursor and connection
cursor.close()
conn.close()

# Display the combined DataFrame
print(df_combined)


       article_id search_platform                   search_keyword  \
0               1          GOOGLE  surgical robot OR medical robot   
1               2          GOOGLE  surgical robot OR medical robot   
2               3          GOOGLE  surgical robot OR medical robot   
3               4          GOOGLE  surgical robot OR medical robot   
4               5          GOOGLE  surgical robot OR medical robot   
...           ...             ...                              ...   
21292       21293           NAVER                           의료용 로봇   
21293       21294           NAVER                           의료용 로봇   
21294       21295           NAVER                           의료용 로봇   
21295       21296           NAVER                           의료용 로봇   
21296       21297           NAVER                           의료용 로봇   

         pub_date                                              title  \
0      2020-01-03                       Orthopedic Surgeon-Inventors   
1      2020-01-

# plotly를 활용한 divergent stacked bars

## 미국 기업별 비교

### Medtronic MDT

In [71]:
# Query data from database
senti_query = "SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = 'MDT'"
df_senti = pd.read_sql(senti_query, conn)

# Close the connection
conn.close()

# Print the DataFrame to verify
print(df_senti.head())


     pub_date sentiment_from_ctext sentiment_from_keyword
0  2020-01-02             positive               positive
1  2020-01-03             positive               positive
2  2020-01-06             positive               positive
3  2020-01-07             negative               negative
4  2020-01-07             negative               negative


/tmp/ipykernel_5217/2910721.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_senti = pd.read_sql(senti_query, conn)


In [76]:
# Initialize counters for sentiments
positive_counts = []
neutral_counts = []
negative_counts = []

# Iterate over rows in the DataFrame
for index, row in df_senti.iterrows():
    # Count positive, neutral, and negative sentiments from both columns
    positive = 0
    neutral = 0
    negative = 0

    # Count sentiments from sentiment_from_ctext column
    if row['sentiment_from_ctext'] == 'positive':
        positive += 1
    elif row['sentiment_from_ctext'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_ctext'] == 'negative':
        negative += 1

    # Count sentiments from sentiment_from_keyword column
    if row['sentiment_from_keyword'] == 'positive':
        positive += 1
    elif row['sentiment_from_keyword'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_keyword'] == 'negative':
        negative += 1

    # Append counts to respective lists
    positive_counts.append(positive)
    neutral_counts.append(neutral)
    negative_counts.append(negative)

# Add counts as new columns in the DataFrame
df_senti['positive'] = positive_counts
df_senti['neutral'] = neutral_counts
df_senti['negative'] = negative_counts

# Group by 'pub_date' and aggregate counts
df_agg = df_senti.groupby('pub_date').sum().reset_index()

# Print aggregated DataFrame to verify
print(df_agg.head())

     pub_date sentiment_from_ctext sentiment_from_keyword  positive  neutral  \
0  2020-01-02             positive               positive         2        0   
1  2020-01-03             positive               positive         2        0   
2  2020-01-06             positive               positive         2        0   
3  2020-01-07     negativenegative       negativenegative         0        0   
4  2020-01-08     negativenegative       positivenegative         1        0   

   negative  
0         0  
1         0  
2         0  
3         4  
4         3  


In [77]:
# Prepare data
dates = df_agg['pub_date']
positive = df_agg['positive']
neutral = df_agg['neutral']
negative = df_agg['negative']

# Create a figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red'
))

# Update layout to create a divergent stacked bar chart
fig.update_layout(
    barmode='relative',
    title='Combined Sentiment Analysis Results',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    template='plotly_white'
)

# Show the plot
fig.show()


In [431]:
company = 'MDT'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)


# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Medtronic PLC ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_5217/1418831329.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### intuitive

In [390]:
company = 'ISRG'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)


# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Intuitive Surgical ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_5217/4202531023.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### Stryker Corporation

In [367]:
company = 'SYK'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)


# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Stryker Corporation ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_5217/1126108231.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### globus

In [369]:
company = 'GMED'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)


# Initialize counters for sentiments
positive_counts = []
neutral_counts = []
negative_counts = []

# Iterate over rows in the DataFrame
for index, row in df_senti.iterrows():
    # Count positive, neutral, and negative sentiments from both columns
    positive = 0
    neutral = 0
    negative = 0

    # Count sentiments from sentiment_from_ctext column
    if row['sentiment_from_ctext'] == 'positive':
        positive += 1
    elif row['sentiment_from_ctext'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_ctext'] == 'negative':
        negative += 1

    # Count sentiments from sentiment_from_keyword column
    if row['sentiment_from_keyword'] == 'positive':
        positive += 1
    elif row['sentiment_from_keyword'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_keyword'] == 'negative':
        negative += 1

    # Append counts to respective lists
    positive_counts.append(positive)
    neutral_counts.append(neutral)
    negative_counts.append(negative)

# Add counts as new columns in the DataFrame
df_senti['positive'] = positive_counts
df_senti['neutral'] = neutral_counts
df_senti['negative'] = negative_counts

# Group by 'pub_date' and aggregate counts
df_agg = df_senti.groupby('pub_date').sum().reset_index()

# Connect to MySQL database and fetch sentiment analysis data (already explained in previous steps)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()


# Prepare data
dates = df_agg['pub_date']
positive = df_agg['positive']
neutral = df_agg['neutral']
negative = df_agg['negative']

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Globus Medical ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    # yaxis=dict(
    #     title='Sentiment Count',
    #     overlaying='y',
    #     side='left',
    #     showgrid=False,
    #     titlefont=dict(
    #         color='black',
    #         size=18
    #     ),
    #     tickfont=dict(
    #         color='black'
    #     )
    # ),
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        ),
        # range=[min(mean_closed_price.min(), df_agg[['positive', 'neutral', 'negative']].min().min()),
        #        max(mean_closed_price.max(), df_agg[['positive', 'neutral', 'negative']].max().max())]
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_5217/3834956296.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### Smith & Nephew

In [388]:
company = 'SNN'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)


# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Smith & Nephew ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()

/tmp/ipykernel_5217/1880381717.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### Johnson & Johnson

In [385]:
company = 'JNJ'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)


# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Intuitive ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=8
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()

/tmp/ipykernel_5217/3015172157.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### Asensus

In [376]:
# Query data from database
senti_query = "SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = 'ASXC'"
df_senti = pd.read_sql(senti_query, conn)

# # Close the connection
# conn.close()

# # Print the DataFrame to verify
# print(df_senti.head())


/tmp/ipykernel_5217/2484180585.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [379]:
# Initialize counters for sentiments
positive_counts = []
neutral_counts = []
negative_counts = []

# Iterate over rows in the DataFrame
for index, row in df_senti.iterrows():
    # Count positive, neutral, and negative sentiments from both columns
    positive = 0
    neutral = 0
    negative = 0

    # Count sentiments from sentiment_from_ctext column
    if row['sentiment_from_ctext'] == 'positive':
        positive += 1
    elif row['sentiment_from_ctext'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_ctext'] == 'negative':
        negative += 1

    # Count sentiments from sentiment_from_keyword column
    if row['sentiment_from_keyword'] == 'positive':
        positive += 1
    elif row['sentiment_from_keyword'] == 'neutral':
        neutral += 1
    elif row['sentiment_from_keyword'] == 'negative':
        negative += 1

    # Append counts to respective lists
    positive_counts.append(positive)
    neutral_counts.append(neutral)
    negative_counts.append(negative)

# Add counts as new columns in the DataFrame
df_senti['positive'] = positive_counts
df_senti['neutral'] = neutral_counts
df_senti['negative'] = negative_counts

# Group by 'pub_date' and aggregate counts
df_agg = df_senti.groupby('pub_date').sum().reset_index()

# Print aggregated DataFrame to verify
print(df_agg.head())

     pub_date sentiment_from_ctext sentiment_from_keyword  positive  neutral  \
0  2020-01-08             positive               positive         2        0   
1  2020-01-14             positive               positive         2        0   
2  2020-01-22             positive               positive         2        0   
3  2020-02-03             negative               positive         1        0   
4  2020-02-13             positive               positive         2        0   

   negative  
0         0  
1         0  
2         0  
3         1  
4         0  


--

In [383]:
import pandas as pd
import mysql.connector
import plotly.graph_objects as go
import yfinance as yf

company = 'ASXC'

# Fetch stock price data
ticker_symbol = company
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data
dates = df_agg['pub_date']
positive = df_agg['positive']
neutral = df_agg['neutral']
negative = df_agg['negative']

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis = 'y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Combined Sentiment Analysis and Mean Closed Price of Asensus Surgical, Inc. ({company})',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis_side='left',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',
)

# Show the plot
fig.show()


[*********************100%%**********************]  1 of 1 completed


## 미국 전체 로봇 비교

In [35]:
company = 'surgical robot OR medical robot'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = ['ISRG', 'SYK', 'MDT', 'GMED', 'ASXC', 'SNN', 'JNJ']
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)['Adj Close']

#stock_dataHandling missing data by forward filling
stock_data.fillna(method='ffill', inplace=True)
# Normalize the data to start from the same point for index calculation
normalized_data = stock_data / stock_data.iloc[0]
# Calculate the equal-weighted sector index
mean_closed_price = normalized_data.mean(axis=1)

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.4,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title='Sentiment Analysis and Mean Closed Price of US Medical/Surgical Robot Sector',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_9699/1151091440.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  7 of 7 completed
/tmp/ipykernel_9699/1151091440.py:14: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



## 한국 기업별 비교

### 고영

In [47]:
company = '고영'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = '098460.KQ'
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.4,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Sentiment Analysis and Mean Closed Price of {company}',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()

/tmp/ipykernel_9699/3321223384.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### 큐렉소

In [49]:
company = '큐렉소'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = '060280.KQ'
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)


# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.4,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Sentiment Analysis and Mean Closed Price of {company}',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()

/tmp/ipykernel_9699/1778446450.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


### 미래컴퍼니

In [50]:
company = '미래컴퍼니'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = '049950.KQ'
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)


# Calculate mean closed price by date
mean_closed_price = stock_data['Adj Close'].groupby(stock_data.index.date).mean()

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.4,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title=f'Sentiment Analysis and Mean Closed Price of {company}',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()

/tmp/ipykernel_9699/2185077413.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  1 of 1 completed


## 한국 전체 로봇 비교

In [32]:
import pandas as pd
import mysql.connector
import plotly.graph_objects as go
import yfinance as yf

company = '의료용 로봇'

# Query data from database
senti_query = f"SELECT a.pub_date, s.sentiment_from_ctext, s.sentiment_from_keyword FROM article a, sentiment s WHERE a.id = s.article_id AND a.search_keyword = '{company}'"
df_senti = pd.read_sql(senti_query, conn)

# Fetch stock price data
ticker_symbol = ['098460.KQ', '060280.KQ', '049950.KQ']
start_date = '2020-01-01'
end_date = '2024-05-01'
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)['Adj Close']

#stock_dataHandling missing data by forward filling
stock_data.fillna(method='ffill', inplace=True)
# Normalize the data to start from the same point for index calculation
normalized_data = stock_data / stock_data.iloc[0]
# Calculate the equal-weighted sector index
mean_closed_price = normalized_data.mean(axis=1)

# Prepare data. count sentiment analysis
dates = df_senti['pub_date']
positive = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'positive' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'positive' else 0)
neutral = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'neutral' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'neutral' else 0)
negative = df_senti['sentiment_from_ctext'].apply(lambda x: 1 if x == 'negative' else 0) + df_senti['sentiment_from_keyword'].apply(lambda x: 1 if x == 'negative' else 0)

# Create Plotly figure
fig = go.Figure()

# Add traces for positive, neutral, and negative sentiments
fig.add_trace(go.Bar(
    x=dates,
    y=positive,
    name='Positive',
    marker_color='green',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='green',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=neutral,
    name='Neutral',
    marker_color='gray',
    width=0.4,
    opacity=0.4,
    marker_line_width=1.5,
    marker_line_color='black',
    offsetgroup=0,
    # yaxis='y2'
))

fig.add_trace(go.Bar(
    x=dates,
    y=-negative,  # Negative sentiment is shown below the baseline
    name='Negative',
    marker_color='red',
    width=0.4,
    opacity=0.8,
    marker_line_width=1.5,
    marker_line_color='red',
    offsetgroup=0,
    # yaxis='y2'
))

# Add trace for mean closed price
fig.add_trace(go.Scatter(
    x=mean_closed_price.index,
    y=mean_closed_price,
    mode='lines',
    name='Mean Closed Price',
    line=dict(color='blue', width=2),
    yaxis='y2'
))

# Update layout to combine both plots and adjust y-axis range
fig.update_layout(
    title='Sentiment Analysis and Mean Closed Price of KR Medical/Surgical Robot Sector',
    xaxis_title='Date',
    yaxis_title='Sentiment Count',
    yaxis2=dict(
        title='Mean Closed Price',
        overlaying='y',
        side='right',
        showgrid=False,
        titlefont=dict(
            color='blue',
            size=10
        ),
        tickfont=dict(
            color='blue',
            size=10
        )
    ),
    template='plotly_white',

)

# Show the plot
fig.show()


/tmp/ipykernel_9699/2197178243.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

[*********************100%%**********************]  3 of 3 completed
/tmp/ipykernel_9699/2197178243.py:19: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

